<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex10-TestingCAPM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Investment Analysis**, Bates, Boyer, Fletcher

# Example Chapter 10: Testing the CAPM
In this notebook we download return data from Ken French's website and estimate the alpha's for each porfolio along with the 95% confidence intervals.   

### Imports and Setup

In [ ]:
!curl -O https://raw.githubusercontent.com/boyerb/Investments/master/functions/simple_finance.py
import simple_finance as sf
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

### Choose Strategy and Estimate Alpha
#### Function Description  
**`get_ff_strategies(stype: start_date`** (optional)**`, end_date`** (optional)**`details`** (optional)**`)-> pd.DataFrame`**     
Imports historical monthly returns for decile portfolios based on trading strategies from Ken French's website and returns the data as a DataFrame  

**Inputs**  
- `stype` (*string*):  strategy type, can be either `'beta'` or `'momentum'`. Other strategies coming soon.  
- `start_date` (*string*, optional):  starting month in format `'yyyy-mm'`  
- `end_date` (*string*, optional): ending month in format `'yyyy-mm'`
- `details`(*bool*, optional, default=`False`):  
  - If True, the function will print out a simple description of the strategy and construction details.  

**Returns**  
- (*DataFrame*): Decile returns for the trading stratgey, the value-weighted market return, and the return on short-term t-bills.
  - *Index*: Monthly date index (`pandas.PeriodIndex` with `freq='M'`).
 - *Decile Returns for Strategy* : Dec 1, Dec 2, Dec 3,...Dec 9, Dec 10
 - *mkt* : The return on the value-weighted portfolio
 - *rf* : The return on short-term t-bills

**Example Usage**:  
1. `dat=sf.get_ff_strategies(stype='momentum')`  
Imports all avaiable monthly returns for the momentum strategy.

2. `dat=sf.get_ff_strategies(stype='beta', start_date='1990-01', end_date=`2024-12`, details=True)`  
Imports monthly returns for the beta strategy from January 1990 to December 2024, and prints out information about the strategy.


In [ ]:
# Choose the strategy
dat=sf.get_ff_strategies(stype='beta',details=True)

dat['exmkt']=dat['mkt']-dat['rf']

# Initialize lists to store intercepts and confidence intervals
intercepts = []
conf_intervals = []

# Loop through each column corresponding to Dec 1 - Dec 10
for column in [f"Dec {i}" for i in range(1, 11)]:
    y = dat[column]-dat['rf']  # Dependent variable (Decile return)
    X = dat['exmkt']  # Independent variable (Market excess return)
    X = sm.add_constant(X)    # Add constant to X for intercept

    # Fit regression model
    model = sm.OLS(y, X).fit()

    # Get the intercept and 95% confidence intervals
    intercept = model.params['const'] * 12  # Multiply intercept by 12
    ci_lower, ci_upper = model.conf_int().iloc[0] * 12  # Multiply confidence intervals by 12

    # Get the intercept and 95% confidence intervals
    intercepts.append(intercept)  # Append intercept to list
    conf_intervals.append([ci_lower, ci_upper])  # Append 95% CI for the intercept

# Convert results to DataFrame for easier manipulation
results_df = pd.DataFrame({
    "Decile": [f"Dec {i}" for i in range(1, 11)],
    "Intercept": intercepts,
    "CI_Lower": [ci[0] for ci in conf_intervals],
    "CI_Upper": [ci[1] for ci in conf_intervals]
})


In [ ]:
# Plot the intercepts with confidence intervals
plt.figure(figsize=(10, 6))
plt.errorbar(results_df["Decile"], results_df["Intercept"],
             yerr=[results_df["Intercept"] - results_df["CI_Lower"],
                   results_df["CI_Upper"] - results_df["Intercept"]],
             fmt='o', capsize=5, capthick=2, label="Intercept ± 95% CI", color='b')

# Labels and title
plt.xlabel("Decile",fontsize=16)
plt.ylabel("Intercept",fontsize=16)
#plt.title("Regression Intercepts with 95% Confidence Intervals")
plt.xticks(rotation=45)  # Rotate x-axis labels for readability
plt.axhline(0, color='black', linestyle='--', linewidth=2)  # Reference line at 0
plt.legend()
# Format y-axis as percentages
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=1))

# Show the plot
plt.show()